## Imports

In [1]:
# imports
import pandas as pd
import altair as alt
from vega_datasets import data
from datetime import datetime
import time


## Read and Manipulate Data

In [2]:
# read in aggregated bike traffic data
df_traffic = (pd.read_excel("data/traffic/cleaned/aggregated-combined-traffic-data.xlsx")).fillna(0)


In [3]:
# rename cols
df_traffic = df_traffic.rename(columns={'YR_2016':'2016', 'YR_2017': "2017", 'YR_2018':'2018'})

# transform data into long form
df_traffic = df_traffic.melt(id_vars=["Intersection", "Time"], value_vars = ["2016","2017","2018"], var_name="Year", 
                    value_name="Traffic")

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic
0,6,01:00 AM,2016,4.571429
1,6,01:00 PM,2016,57.142857
2,6,02:00 AM,2016,2.928571
3,6,02:00 PM,2016,60.642857
4,6,03:00 AM,2016,1.714286
...,...,...,...,...
283,1071,10:00 PM,2018,4.500000
284,1071,11:00 AM,2018,8.000000
285,1071,11:00 PM,2018,1.500000
286,1071,12:00 AM,2018,1.000000


In [4]:
# add latitude and longitude column to df_traffic based on intersection
df_traffic['Latitude'] = df_traffic['Intersection'].map({6: '42.351986', 451: '42.340097', 1071: '42.32693', 601: '42.33693'})
df_traffic['Longitude'] = df_traffic['Intersection'].map({6: '-71.090079', 451: '-71.081539', 1071: '-71.06684', 601: '-71.07806'})

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic,Latitude,Longitude
0,6,01:00 AM,2016,4.571429,42.351986,-71.090079
1,6,01:00 PM,2016,57.142857,42.351986,-71.090079
2,6,02:00 AM,2016,2.928571,42.351986,-71.090079
3,6,02:00 PM,2016,60.642857,42.351986,-71.090079
4,6,03:00 AM,2016,1.714286,42.351986,-71.090079
...,...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,42.32693,-71.06684
284,1071,11:00 AM,2018,8.000000,42.32693,-71.06684
285,1071,11:00 PM,2018,1.500000,42.32693,-71.06684
286,1071,12:00 AM,2018,1.000000,42.32693,-71.06684


In [5]:
# create column for hour of day
df_traffic['Hour'] = df_traffic['Time'].str.split(':').str[0].astype(int)

# 12 AM is 0
df_traffic.loc[df_traffic['Time'].str.contains('12:'), 'Hour'] = 0

# add 12 if PM
df_traffic.loc[df_traffic['Time'].str.contains('PM'), 'Hour'] = df_traffic['Hour'] + 12

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic,Latitude,Longitude,Hour
0,6,01:00 AM,2016,4.571429,42.351986,-71.090079,1
1,6,01:00 PM,2016,57.142857,42.351986,-71.090079,13
2,6,02:00 AM,2016,2.928571,42.351986,-71.090079,2
3,6,02:00 PM,2016,60.642857,42.351986,-71.090079,14
4,6,03:00 AM,2016,1.714286,42.351986,-71.090079,3
...,...,...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,42.32693,-71.06684,22
284,1071,11:00 AM,2018,8.000000,42.32693,-71.06684,11
285,1071,11:00 PM,2018,1.500000,42.32693,-71.06684,23
286,1071,12:00 AM,2018,1.000000,42.32693,-71.06684,0


In [6]:
# get info on dataframe
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Intersection  288 non-null    int64  
 1   Time          288 non-null    object 
 2   Year          288 non-null    object 
 3   Traffic       288 non-null    float64
 4   Latitude      288 non-null    object 
 5   Longitude     288 non-null    object 
 6   Hour          288 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 15.9+ KB


## Bar Chart

In [7]:
# create selection
select = alt.selection(type='multi', on='click', nearest=False, fields=['Year'])

# create slider
dropdown = alt.binding_select(options=['12:00 AM', '01:00 AM', '02:00 AM', '03:00 AM', '04:00 AM', '05:00 AM', 
                                       '06:00 AM', '07:00 AM', '08:00 AM', '09:00 AM', '10:00 AM', '11:00 AM', 
                                       '12:00 PM', '01:00 PM', '02:00 PM', '03:00 PM', '04:00 PM', '05:00 PM', 
                                       '06:00 PM', '07:00 PM', '08:00 PM', '09:00 PM', '10:00 PM', '11:00 PM'])
select_time = alt.selection_single(name="Time", fields=['Time'], bind=dropdown, init={'Time': '12:00 AM'})

# create chart 
bar_chart = alt.Chart(df_traffic).mark_bar().encode(
    x='sum(Traffic):Q',
    y='Year:N',
    color=alt.condition(select, alt.Color('Year:N', scale=alt.Scale(scheme='dark2'), legend=alt.Legend(title='Year', 
    
        # legend configuration based on final chart
        orient='none',
        legendX=780, legendY=500,
        direction='vertical',
        titleAnchor='middle')), alt.value('lightgray'))
).properties(
    height=200, 
    width=600,
    title='Hourly Bike Traffic by Year'
).add_selection(
    select
).add_selection(
    select_time
).transform_filter(
    select_time
)

# show chart
bar_chart

alt.Chart(...)

In [8]:
df_traffic

,Intersection,Time,Year,Traffic,Latitude,Longitude,Hour
0,6,01:00 AM,2016,4.571429,42.351986,-71.090079,1
1,6,01:00 PM,2016,57.142857,42.351986,-71.090079,13
2,6,02:00 AM,2016,2.928571,42.351986,-71.090079,2
3,6,02:00 PM,2016,60.642857,42.351986,-71.090079,14
4,6,03:00 AM,2016,1.714286,42.351986,-71.090079,3
...,...,...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,42.32693,-71.06684,22
284,1071,11:00 AM,2018,8.000000,42.32693,-71.06684,11
285,1071,11:00 PM,2018,1.500000,42.32693,-71.06684,23
286,1071,12:00 AM,2018,1.000000,42.32693,-71.06684,0


In [9]:
# convert time to datetime
df_traffic['Time_form'] = pd.to_datetime(df_traffic['Time'], format='%I:%M %p')

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic,Latitude,Longitude,Hour,Time_form
0,6,01:00 AM,2016,4.571429,42.351986,-71.090079,1,1900-01-01 01:00:00
1,6,01:00 PM,2016,57.142857,42.351986,-71.090079,13,1900-01-01 13:00:00
2,6,02:00 AM,2016,2.928571,42.351986,-71.090079,2,1900-01-01 02:00:00
3,6,02:00 PM,2016,60.642857,42.351986,-71.090079,14,1900-01-01 14:00:00
4,6,03:00 AM,2016,1.714286,42.351986,-71.090079,3,1900-01-01 03:00:00
...,...,...,...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,42.32693,-71.06684,22,1900-01-01 22:00:00
284,1071,11:00 AM,2018,8.000000,42.32693,-71.06684,11,1900-01-01 11:00:00
285,1071,11:00 PM,2018,1.500000,42.32693,-71.06684,23,1900-01-01 23:00:00
286,1071,12:00 AM,2018,1.000000,42.32693,-71.06684,0,1900-01-01 00:00:00


In [10]:
# add intersection name column
df_traffic['Intersection_name'] = df_traffic['Intersection'].map({6: 'Massachusetts Ave & Back St', 451: 'Massachusetts Ave & Columbus Ave', 1071: 'Massachusetts Ave & Newmarket Sq', 601: 'Massachusetts Ave & Shawmut Ave'})

In [11]:
selection = alt.selection_multi(fields=['Year'], bind='legend')

# line chart of bike traffic by hour of day
line_chart = alt.Chart(df_traffic).mark_line().encode(
    x=alt.X('Time_form:T', axis=alt.Axis(title='Time of Day', labelAngle=45, tickCount=24)),
    y=alt.Y('mean(Traffic):Q', axis=alt.Axis(title='Bike Traffic')),
    color=alt.Color('Year:N', scale=alt.Scale(scheme='dark2'), legend=alt.Legend(title='Year',  orient='none', legendX=820, legendY=500, direction='vertical', titleAnchor='middle')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    height=200,
    width=700,
    title='Hourly Bike Traffic by Year'
).add_selection(
    selection
)

# add selection at one x value
nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=['Time_form'], empty='none')

points = alt.Chart(df_traffic).mark_point().encode(
    x=alt.X('Time_form:T', axis=alt.Axis(title='Time of Day', labelAngle=45, tickCount=24)),
    y=alt.Y('mean(Traffic):Q', axis=alt.Axis(title='Bike Traffic')),
    color=alt.Color('Year:N', scale=alt.Scale(scheme='dark2'), legend=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).properties(
    height=200,
    width=700,
    title='Hourly Bike Traffic by Year'
).transform_filter(
    selection
)

selectors = alt.Chart(df_traffic).mark_point().encode(
    x='Time_form:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

axis_labels = (
    "datum.label == 1900 ? '12 AM' : datum.label"
)

text = alt.Chart(df_traffic).mark_text(align='left', dx=5, dy=-5).encode(
    x=alt.X('Time_form:T', axis=alt.Axis(title='Time of Day', labelExpr=axis_labels, labelAngle=45, tickCount=24)),
    y=alt.Y('mean(Traffic):Q', axis=alt.Axis(title='Bike Traffic')),
    color=alt.Color('Year:N', scale=alt.Scale(scheme='dark2'), legend=None),
    text=alt.condition(nearest, 'mean(Traffic):Q', alt.value(' '), format='.2f')
).properties(
    height=200,
    width=700,
    title='Hourly Bike Traffic by Year'
).transform_filter(
    selection
)

rules = alt.Chart(df_traffic).mark_rule(color='gray').encode(
    x=alt.X('Time_form:T', axis=alt.Axis(title='Time of Day', labelAngle=45, tickCount=24)),
).transform_filter(
    nearest
)

# show chart
line = line_chart + points + selectors + text + rules

line

alt.LayerChart(...)

## Background Map Image

In [12]:
# define source df for altair to use
source_bike_network = pd.DataFrame.from_records([
    {"x":  0, "y": 0,
     "img": "https://raw.githubusercontent.com/lsouth/DS4200/main/bike_lanes_2.jpeg"}
])

# altair chart for bike network background
bike_network = alt.Chart(source_bike_network).mark_image(
    width=500,
    height=500
).encode(
    x='x',
    y='y', 
    url='img'
)

# show chart
bike_network

alt.Chart(...)

## Scatterplot of Bike Counts

In [13]:
# make scatter plot of bike counts at each intersection
traffic = alt.Chart(df_traffic).transform_filter(
    selection # filters the result based on bar chart selection
).transform_filter(
    nearest # filters the result based on time selection
).transform_aggregate(
    traffic='mean(Traffic)',
    groupby=['Latitude', 'Longitude', 'Intersection_name']
).mark_circle(
    fill='red', # make all circles red
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size=alt.Size('traffic:Q', scale=alt.Scale(range=[0, 500]), legend=alt.Legend(title='Bike Count', 
        # legend configuration based on final chart
        orient='none',
        legendX=425, legendY=-80,
        direction='horizontal',
        titleAnchor='middle')),
    tooltip=[alt.Text('traffic:Q', format=',.2f', title="Average Number of Bikes"), alt.Text('Intersection_name:N', title="Intersection")]
)

## Layer Plot on Map

In [14]:
# combine bike network map image and bike traffic scatter plot
traffic_map = bike_network + traffic

# add title
traffic_map = traffic_map.properties(
    title='Average of Daily Bike Traffic at Each Intersection'
)


## Bike Lane Legend

In [15]:
# define source df of bike lanes legend for altair to use
source_bike_legend = pd.DataFrame.from_records([
    {"x":  0, "y": 0,
     "img": "https://raw.githubusercontent.com/lsouth/DS4200/main/bike_lane_legend_full.png"}
])

# altair chart for bike lanes legend
bike_network_legend = alt.Chart(source_bike_legend).mark_image(
    width=400,
    height=400
).encode(
    x=alt.X('x', axis=None),
    y=alt.Y('y', axis=None), 
    url='img'
)

# show chart
bike_network_legend

alt.Chart(...)

## Concatenate for Final Static Chart

In [16]:
# hconcatenate legend with bike traffic map chart
final = alt.hconcat(bike_network_legend, traffic_map)

# vconcat with bar chart and remove axis behind legend
final = alt.vconcat(final, line, center=True).configure_view(strokeWidth=0)

# show chart
final

alt.VConcatChart(...)

In [18]:
# save chart as html file
final.save('vis.html')